In [1]:
import salabim as sim

In [4]:
class Car(sim.Component):
    def process(self):
        while True:
            self.hold(1)

env = sim.Environment(trace=True)
Car()
env.run(till=5)

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               3468253836.py
    6                                  default environment initialize       
    6                                  main create                          
    6       0.000 main                 current                              
    7                                  car.1 create                         
    7                                  car.1 activate                       scheduled for 0.000 @    2+ process=process
    8                                  main run +5.000                      scheduled for 5.000 @    8+
    2+      0.000 car.1                current                              
    4                                  car.1 hold +1.000                    scheduled fo